<a id='top'></a>

# CSCI 3022: Intro to Data Science - Spring 2020 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Friday May 1**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**: Jeremy M. Hein 

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.


---

In [84]:
from scipy import stats
from math import isnan
import numpy as np 
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline
from calendar import month_name, different_locale

<br>

---

### [50 points] Problem 1: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [215]:
#load file into pandas dataframe
file_path = "houses.csv"
df = pd.read_csv(file_path)

#drop any rows with missing data
df = df.dropna(how='any')

#print datafram
df

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,118.94,2113.0,4,7.1,3.23,7.0,3.0,1.00,0.82,12.99,273.0,0.123131
1,20.60,1773.0,2,7.4,4.19,5.0,5.0,1.00,0.99,16.77,184.0,-0.200243
2,260.84,2511.0,3,7.0,3.38,2.0,0.0,1.00,1.17,16.49,141.0,0.258651
3,101.33,2000.0,2,7.9,4.05,6.0,8.0,0.13,0.92,8.28,146.0,0.147983
4,96.94,1476.0,1,7.5,5.75,4.0,1.0,1.00,1.73,5.90,178.0,-0.113756
...,...,...,...,...,...,...,...,...,...,...,...,...
67,237.66,1403.0,2,6.6,5.97,8.0,2.0,1.00,2.16,6.85,271.0,0.352572
68,65.39,1880.0,4,4.1,3.59,7.0,2.0,0.60,1.67,16.83,211.0,0.331913
69,56.24,1996.0,2,7.4,3.38,4.0,4.0,1.00,0.71,6.42,215.0,0.627473
70,4.87,1732.0,1,7.1,3.96,7.0,2.0,1.00,1.46,19.44,122.0,0.411308


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

**Hypothesis Setup:**

Since we're testing if there is at least one feature realted to the response in y, we will use the F-test for MLR.  (Note that if we were to test each feature individually, we would run into the Problem of Multiple Comparisons. Thus, the F-test is appropriate here).  Our hypotheses are as follows:

$H_{0}: \beta_{0} = \beta_{1} = \beta_{2} = .... = \beta_{p} = 0$ (None of our features are related to to the haunted score)

$H_{1}: \beta_{k} \neq 0$ for at least one value of k in 1, 2, 3, ... , p (At least one feature is realted to the haunted score)

Consistent with the F-test, if our F-statistic is greater than 1 and our p-value is less than alpha (0.01, in this case), we can conlude there is sufficient evidence that at least one feature is related to our haunted score and reject the null hypothesis.

In [216]:
# Features in 2D array
X = df[["age", "area", "bathrooms", "distance metro", "distance cemetery", "cats", "howls", "clouds", "precipitation", "misery index", "ice cream sold"]]

# Add a constant to the array for the intecept 
X = sm.add_constant(X)

# Response data 
y = df["haunted"]

# Fit OLS model 
model = sm.OLS(y, X).fit() 

# Print model summary 
model.summary()

C:\Users\jerem\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                haunted   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     16.22
Date:                Fri, 01 May 2020   Prob (F-statistic):           3.75e-14
Time:                        12:19:42   Log-Likelihood:                 4.2800
No. Observations:                  72   AIC:                             15.44
Df Residuals:                      60   BIC:                             42.76
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3097      0.342      0.905      0.369      -0.375       0.994
age                   0.0012      0.000      3.267      0.002       0.000       0.002
area               3.573e-05    8.1e-05      0.441      0.661      -0.000       0.000
bathrooms            -0.0104      0.031     -0.330      0.743      -0.073       0.052
distance metro       -0.0173      0.036     -0.477      0.635      -0.090       0.055
distance cemetery    -0.1149      0.010    -11.107      0.000      -0.136      -0.094
cats                  0.0370      0.011      3.343      0.001       0.015       0.059
howls                -0.0091      0.013     -0.694      0.491      -0.035       0.017
clouds               -0.1022      0.139     -0.738      0.463      -0.379       0.175
precipitation         0.0166      0.076      0.217      0.829      -0.136       0.169
misery index          0.0051      0.006      0.802      0.426      -0.008       0.018
ice cream sold        0.0004      0.001      0.788      0.434      -0.001       0.002
==============================================================================
Omnibus:                        0.027   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.986   Jarque-Bera (JB):                0.176
Skew:                          -0.026   Prob(JB):                        0.916
Kurtosis:                       2.764   Cond. No.                     2.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Results:**

We can see from our model that our F-statistic is 16.22 and our p-value is less than 0.01, indicating we have sufficient evidence to reject the null hypothesis and conlude that at least one of our features is related to the haunted score.

**Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

In [217]:
#define feature column names
features = ["age", "area", "bathrooms", "distance metro", "distance cemetery", "cats", "howls", "clouds", "precipitation", "misery index", "ice cream sold"]

#define list for features we select in our function
f_select = []

def forward_select(df, resp_str, maxk):
    #define our y value
    y = df[resp_str]
    
    #number of features we have selected
    count = 0 
    
    #loop until we have 5 added features
    while count < maxk:
        
        #SLR for each feature (p) we have in our dataframe
        if count == 0: 

            #calculate the first SLR
            X = df[features[0]]
            X = sm.add_constant(X)
            model = sm.OLS(y,X).fit()

            #set current SSE and lowest SSE to first SLR
            SSEcurr = model.ssr
            SSElow = model.ssr
            SSEname = features[0]

            #now calculate SSE for each feature, and replace low SSE
            for m in features:
                X = df[[m]] #reset X feature
                X = sm.add_constant(X)
                model = sm.OLS(y,X).fit() #fit model with new X feature
                SSEcurr = model.ssr #set current SSE
                if SSEcurr < SSElow: #if current is less than low
                    SSEname = m 
                    SSElow = SSEcurr

            features.remove(SSEname) #remove name from features
            f_select.append(SSEname) #add name to selected features
            count += 1 #add 1 to the count
            
            #print additions to list
            print(SSEname, " has been added to features")
            print("Current features: ", f_select)
        
        #MLR for remaining features (p) we have in our dataframe
        if count != 0: 

            #loop through features
            for m in features:
                f_select.append(m) #add to list
                X = df[f_select] #reset x to be previous added + m
                f_select.remove(m) #remove m so we don't lose track of added features
                X = sm.add_constant(X)
                model = sm.OLS(y,X).fit() #fit model with new X feature
                SSEcurr = model.ssr #set current SSE

                if SSEcurr < SSElow:
                    SSEname = m
                    SSElow = SSEcurr

            features.remove(SSEname) #remove name from features
            f_select.append(SSEname) #add name to selected features
            count += 1 #add 1 to the count
            
            print(SSEname, " has been added to features")
            print("Current features: ", f_select)
                      
forward_select(df, 'haunted', 5)

distance cemetery  has been added to features
Current features:  ['distance cemetery']
cats  has been added to features
Current features:  ['distance cemetery', 'cats']
age  has been added to features
Current features:  ['distance cemetery', 'cats', 'age']
misery index  has been added to features
Current features:  ['distance cemetery', 'cats', 'age', 'misery index']
distance metro  has been added to features
Current features:  ['distance cemetery', 'cats', 'age', 'misery index', 'distance metro']


**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

In [218]:
y = df['haunted']
X = df[f_select]
X = sm.add_constant(X)
model_reduced = sm.OLS(y,X).fit() #fit model with new X feature

model_reduced.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                haunted   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     38.12
Date:                Fri, 01 May 2020   Prob (F-statistic):           3.40e-18
Time:                        12:19:51   Log-Likelihood:                 3.4850
No. Observations:                  72   AIC:                             5.030
Df Residuals:                      66   BIC:                             18.69
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3848      0.240      1.607      0.113      -0.093       0.863
distance cemetery    -0.1114      0.009    -12.215      0.000      -0.130      -0.093
cats                  0.0348      0.010      3.398      0.001       0.014       0.055
age                   0.0012      0.000      3.529      0.001       0.001       0.002
misery index          0.0058      0.006      0.962      0.339      -0.006       0.018
distance metro       -0.0269      0.032     -0.830      0.409      -0.092       0.038
==============================================================================
Omnibus:                        0.218   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.897   Jarque-Bera (JB):                0.410
Skew:                          -0.034   Prob(JB):                        0.815
Kurtosis:                       2.636   Cond. No.                     1.38e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**MLR Model Including Estimated Parameters:**

Our estimated model will be in the form $\hat{y} = \hat{\beta}_{0} + \hat{\beta}_{dc}x_{dc} + \hat{\beta}_{cats}x_{cats} +\hat{\beta}_{age}x_{age} +\hat{\beta}_{mi}x_{mi} +\hat{\beta}_{dm}x_{dm}$ where dc is distance cemetery, mi is misery index, and dm distance metro.

Using our estimated parameters from our model, we have:

$\hat{y} = 0.3848 + -0.1114x_{dc} + 0.0348x_{cats} + 0.0012x_{age} +0.0058x_{mi} + -0.0269x_{dm}$

**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

**Hypothesis Setup:**

We will use a partial F-test to determine whether there is a statistically significant difference between the full model with all features and the reduced model.  Our hypotheses will be as follows:

$H_{0}:  \beta_{area} = \beta_{bathrooms} = \beta_{howls} = \beta_{clouds} = \beta_{precip} = \beta_{icecreamsold} = 0$ (None of these features are related to to the haunted score)

$H_{1}:$  At least one $\beta$ above $\neq 0$

$SSE_{full}$ = variation unexplained by the full model

$SSE_{reduced}$ = variation unexplained by the reduced model

Our appropriate F-statistic should depend on the difference between SSE's for each model.  In our case, variables we need to calculate the F-statistic are as follows:

p = 11

k = 5

n = 72

We will also get the SSE's for each model using statsmodels.api built-in functions.

In [219]:
p = 11
k = 5
n = 72

# Get SSE's from each model
SSE_full = model.ssr
SSE_reduced = model_reduced.ssr

#calculate F-statistic
f_stat = ((SSE_reduced - SSE_full)/(p-k)) / (SSE_full/(n-p-1))

print(f_stat)

f_alpha = stats.f.ppf(1-(0.05/2),(p-k),(n-p-1)) #95% confidence interval
print(f_alpha)

0.2232889261219416
2.627369592102272


**Results:**

We can see from our results that $F_{stat} < F_{alpha}$, therefore we fail to reject the null hypothesis and conlude that our reduced model is statistically better than the full model. 

**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted house hazard when the following features are observed: 

- $\texttt{age}$: 150 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 5 miles
- $\texttt{cats}$: 20 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
We- $\texttt{ice cream sold}$: 125

We can use the following formula to predict our value based on the observed features above:

$\hat{y} = 0.3848 + -0.1114x_{dc} + 0.0348x_{cats} + 0.0012x_{age} +0.0058x_{mi} + -0.0269x_{dm}$

As computed below, we can see the resulting haunted score is 0.09

In [235]:
# Your code here.

yhat = 0.3848 - 0.1114*5 + 0.0348*20 + 0.0012*150 + 0.0058*10 - 0.0269*25
print(yhat)

0.08930000000000005


**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

So using Govenor Polis's information and combining it with our knowledge of logistic regression, the probability of a house being haunted is:

$$p(\text{haunted} \mid x_1, x_2, \ldots) = \dfrac{1}{1+\texttt{exp}[-(\beta_0 + \beta_1x_1 + \ldots + \beta_p x_p)]}$$

Perform this computation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

In [ ]:
# Your code here.

<br>

---



### [50 points] Problem 2: Amazon Forest Fires
A non-profit trying to protect the amazon rain forest has recruited you to join their data science corps. For your first task, they've given you a dataset with the number of reported forest fires in each state in the Amazon region of Brazil during each month between 1998 and 2017. The Brazilian government has 500 extra wildland firefighters and they have asked your non-profit to determine which state or states they should allocate these firefighters to during each month of the year. To do this, they want you to calculate an 80% confidence interval for the mean and median number of fires that occur during each month for each state, and use those statistics to determine where the firefighters should be assigned.

#### Part A: Loading The CSV
Read the csv located in `amazon.csv` into a pandas data frame. Brazil and many other countries use the period (.) symbol as a thousands separator and a comma (,) as the decimal separator. Ex. One Thousand And $\frac{75}{100}$ would be represented as $1.000,75$ instead of the familiar english notation $1,000.75$. When you read it in, you'll need to use a period(.) as the thousands separator and a comma(,) as the decimal separator. Because the comma is already in use as the decimal separator, this file uses a different character to separate columns in the data. Open up the file in a text editor and figure out what character was used. Then find the correct arguments to `pd.read_csv` to read in this file properly. Look up the docs if you're unsure what the arguments you'll need are. Print out the `.info` summary of the dataframe after you've read it in.

In [178]:
file_path = "amazon.csv"
amazon_fires = pd.read_csv(file_path, sep=';', thousands='.', decimal=',')
amazon_fires

#https://stackoverflow.com/questions/22137723/convert-number-strings-with-commas-in-pandas-dataframe-to-float

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01
...,...,...,...,...,...
6449,2012,Tocantins,Dezembro,128.0,2012-01-01
6450,2013,Tocantins,Dezembro,85.0,2013-01-01
6451,2014,Tocantins,Dezembro,223.0,2014-01-01
6452,2015,Tocantins,Dezembro,373.0,2015-01-01


#### Part B: Data Cleaning

This dataset isn't paticularly useful in it's current state, so we'll need to clean it up a bit. Some data scientists say that most of their job is to wrangle data, so this will give you a taste of cleaning a real world data set. Perform the following tasks. 
1. Drop the 'date' column. The only information this column holds is the year, which we already have in another column. Use the `.info` summary provided to check your work.
2. Drop any rows with null values in any of the remaining columns. Use the provided code to print the number of rows remaining after this step.
3. Print all the unique values of the 'month' column. You'll notice that one is encoded with a different character encoding then the format that pandas is using.
4. Convert the Portugese month names to English month names. If you'd like to use them, we've included the 'month_name' and the 'different_encoding' modules of the python calendar library. There are many ways to accomplish this task, and these modules are not required, but may make things easier. As part of this step, you should make sure that the Portugese month with the encoding problem is translated to the correct english month. Use the `.unique` method provided for you to check your work. 
5. Check the number column for any values that seem impossible. If you find any values you think are impossible, drop them. As a guidline, we would never expect a single state to have more than 50,000 reported forest fires in a single month. Also keep in mind that we are tracking forest fires here. Do negative or fractional forest fires really make sense? You should check for any obivously impossible conditions that you think might occur, and drop rows accordingly. Use the provided code to print the number of rows remaining after this step.
6. Since you're new on the job, some of your co-workers may have played a prank on you... Print out all the unique values of the 'year' column and drop any rows with values that don't make sense. Use the provided code to print the number of rows remaining after this step.
7. For every state in the data, print the number of rows the state has associated with it. A number of states have far more observations than the others. Each state should have roughly 240 observations (20 years multiplied by 12 months/year minus any bad data). Drop all the observations for any states that have more than 240 rows associated with them. For two points of extra credit, figure out why these states have way more rows associated with them than they should. If you choose to do the extra credit, put your answer in the markdown cell below. 
8. To give you an idea of whether your answer is correct, we've provided a unit test below the last cell. It should pass. If it doesn't, go back and figure out which step has gone awry.

We've given you a code cell for each task to make organizing the grading a bit easier. Please perform step 1 in the first code cell and so on.

**NOTE:** Since some of these tasks are not totally trivial, you may use any resources other than your classmates on this part of this problem. This means you may consult google, stack overflow, the python/pandas documentation, some random book on pandas you might have, etc... But you may not consult your classmates for help. We will also be more helpful on this problem in office hours and in response to your *private* piazza messages.  ***CITE ALL RESOURCES USED IN A CODE COMMENT. A URL OR A BOOK TITLE IS SUFFICIENT. ANY CODE OBIVOUSLY COPIED FROM OUTSIDE SOURCES WITH OUT A CITATION WILL EARN YOU NO CREDIT ON THIS PROBLEM.***

In [179]:
#Code for data cleaning task 1 here:
amazon_fires = amazon_fires.drop(columns = ['date'])
amazon_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 4 columns):
year      6453 non-null object
state     6452 non-null object
month     6454 non-null object
number    6448 non-null float64
dtypes: float64(1), object(3)
memory usage: 201.8+ KB


In [180]:
#Code for data cleaning task 2 here:
#drop any rows with missing data
amazon_fires = amazon_fires.dropna(how='any')
print(len(amazon_fires))

6446


In [181]:
#Code for data cleaning task 3 here:
print(amazon_fires['month'].unique())

['Janeiro' 'Fevereiro' 'Mar�o' 'Abril' 'Maio' 'Junho' 'Julho' 'Agosto'
 'Setembro' 'Outubro' 'Novembro' 'Dezembro']


In [182]:
#Code for data cleaning task 4 here:
d = {u'Janeiro':'January', u'Fevereiro':'February', u'Mar�o':'March', u'Abril':'April', u'Maio':'May', u'Junho':'June', u'Julho':'July', u'Agosto':'August', u'Setembro':'September', u'Outubro':'October', u'Novembro':'November', u'Dezembro':'December'}

amazon_fires['month'] = amazon_fires['month'].replace(d, regex=True)

print(amazon_fires['month'].unique())

#reference stackoverflow
#https://stackoverflow.com/questions/37114683/pandas-replace-months-names-within-dataframe-values

['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


In [183]:
#Code for data cleaning task 5

#drop any with negative fires or fires between 0 and 1 (doesn't make sense)
amazon_fires = amazon_fires.loc[(amazon_fires.number == 0.0) | (amazon_fires.number >= 1.0)] 

#drop any with more than 50,000 fires (per directions)
amazon_fires = amazon_fires.loc[amazon_fires.number <= 50000]

#multi-step to drop fractional values

#first add a column that rounds number down and converts to type int
amazon_fires['rounded'] = amazon_fires['number'].apply(np.floor).astype(int)

#add another column that divides number by rounded
amazon_fires['div'] = amazon_fires.number / amazon_fires.rounded

#replace NaN with 1.0 (anything / 0 will result in NaN and we want to keep 0 values in df)
amazon_fires['div'] = amazon_fires['div'].replace(np.nan,1.0)

#keep non-decimal values
amazon_fires = amazon_fires.loc[amazon_fires['div'] == 1.0] 

print(len(amazon_fires))

6441


In [184]:
#Code for data cleaning task 6
print(amazon_fires['year'].unique())

#drop non-numeric values
amazon_fires = amazon_fires[amazon_fires.year.apply(lambda x: x.isnumeric())]
print(amazon_fires['year'].unique())

print(len(amazon_fires))

#https://stackoverflow.com/questions/33961028/remove-non-numeric-rows-in-one-column-with-pandas

['1998' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007'
 '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017'
 '1000bc' '-40' '10bc' "our new data scientist won't notice this"]
['1998' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007'
 '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017']
6437


In [185]:
#Code for data cleaning task 7

#get all unique states
states = amazon_fires['state'].unique()

for s in states:
    length = len(amazon_fires[(amazon_fires['state'] == s)]) ## get the lenght of rows associated with state "s"
    print(s, length) # print the states and number of rows
    if length > 240: # drop if greater than 240
        amazon_fires = amazon_fires.loc[amazon_fires['state'] != s] 


Acre 239
Alagoas 240
Amapa 239
Amazonas 239
Bahia 239
Ceara 239
Distrito Federal 239
Espirito Santo 238
Goias 238
Maranhao 239
Mato Grosso 476
Minas Gerais 237
Para 235
Paraiba 474
Pernambuco 239
Piau 239
Rio 715
Rondonia 239
Roraima 239
Santa Catarina 238
Sao Paulo 239
Sergipe 239
Tocantins 239


In [186]:
#Given Test. This Cell Should Not Throw an Exception!
assert \
    len(amazon_fires['state'].unique()) == 20 and \
    list(amazon_fires['month'].unique()) == \
        ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August','September', 'October', 'November',
             'December'] and \
    len(amazon_fires) == 4772, 'somethings wrong in problem 1.'

#### Part C: Medians and Means!
In this part of the problem, we'll calculate an 80% confidence interval for both the mean and median number of wildfires each state has during each month of the year. 

For the mean you should use the appropriate confidence interval with the correct distribution. Remember to check how many observations we have. Use the sample standard deviation. 

For the median, we'll have to bootstrap it because the median is not known to be normally distributed. You should bootstrap 1000 samples of the same length as the original sample for each month for each state. Calculate the median for each bootstrapped sample. Then take the middle 80% of the bootstrapped medians as your confidnce interval. This is called a bootstrapped percentile median. There are a few more complex and slightly more rigourous ways to estimate the median from bootstrapped samples, but this will serve for our purposes.

You're given a dictionary of dictionaries to store your confidence intervals for the medians and means in. 

Take a look at the dictionary structure below. 

The idea here is that for every month, for every state, you will fill in the `mean_CI` with a length two list that contains the low and high end of the confidence interval for the true mean number of fires for that state in that month. 

Similiarly, for every month, for every state, you will fill in the `median_CI` with a length two list that contains the low and high end of the confidence interval for the true median number of fires for that state in that month.

For example:

When you're done `months['January']['Acre']['mean_CI']` should be a list with the low and high bounds for the confidence interval of the true mean number of wildfires in the state of Acre in January. So `months['January']['Acre']['mean_CI'][0]` should be the low end of the CI for the mean, and `months['January']['Acre']['mean_CI'][1]` should be the high end of the CI for the mean.

`months['January']['Acre']['median_CI']` should hold the confidence interval for the true median number of wildfires in the state of Acre in January. So `months['January']['Acre']['median_CI'][0]` should be the low end of the CI for the median, and `months['January']['Acre']['median_CI'][1]` should be the high end of the CI for the median.

In [187]:
#GIVEN CODE DO NOT CHANGE THIS!!!
#YOU SHOULD BE WRITING CODE IN THE NEXT CELL(s) THAT FILLS IN THE 'months' DICTIONARY.

#If you're curious what copy and deep copy do and why we used them here see an explanation 
#here: https://thispointer.com/python-how-to-copy-a-dictionary-shallow-copy-vs-deep-copy/

from copy import deepcopy

mean_median_dict ={
    'mean_CI' : None,
    'median_CI': None
}

CI_median_num_fires = {
    'Acre': dict(mean_median_dict),
    'Alagoas':dict( mean_median_dict),
    'Amapa':dict( mean_median_dict),
    'Amazonas':dict( mean_median_dict),
    'Bahia':dict( mean_median_dict),
    'Ceara':dict( mean_median_dict),
    'Distrito Federal':dict( mean_median_dict),
    'Espirito Santo':dict( mean_median_dict),
    'Goias':dict( mean_median_dict),
    'Maranhao':dict( mean_median_dict),
    'Minas Gerais':dict( mean_median_dict),
    'Para':dict( mean_median_dict),
    'Pernambuco':dict( mean_median_dict),
    'Piau':dict( mean_median_dict),
    'Rondonia':dict( mean_median_dict),
    'Roraima':dict( mean_median_dict),
    'Santa Catarina':dict( mean_median_dict),
    'Sao Paulo':dict( mean_median_dict),
    'Sergipe':dict( mean_median_dict),
    'Tocantins':dict( mean_median_dict)  
}

months = {
    'January': deepcopy(CI_median_num_fires),
    'February': deepcopy(CI_median_num_fires),
    'March': deepcopy(CI_median_num_fires), 
    'April': deepcopy(CI_median_num_fires), 
    'May': deepcopy(CI_median_num_fires),
    'June': deepcopy(CI_median_num_fires),
    'July': deepcopy(CI_median_num_fires),
    'August': deepcopy(CI_median_num_fires), 
    'September': deepcopy(CI_median_num_fires), 
    'October': deepcopy(CI_median_num_fires),
    'November': deepcopy(CI_median_num_fires),
    'December': deepcopy(CI_median_num_fires)
}

**CI for Mean:**

Each month has at max 20 observations per state, which means we have n < 30 for our sample population size for any given state, in any given month.  Thus, we must use the t-distribution in calculating our mean CI's for each state.

In [188]:
#CALCULATION OF MEAN CI's

#get all unique states and months in an array, respectively
states = amazon_fires['state'].unique()
mnths = amazon_fires['month'].unique()

#loop through each state
for s in states:
    tempDF = amazon_fires.loc[amazon_fires['state'] == s] #get a temp dataframe only 
    
    #loop through each month
    for m in mnths:
        monthly_data = tempDF.loc[tempDF['month'] == m] #get only data related to month
        n = len(monthly_data[(monthly_data['month'] == m)]) #get the length of the dataframe
        xbar = monthly_data['number'].mean(axis=0) #calculate xbar
        stdv = monthly_data['number'].std(axis=0) #calculate sample standard deviation
        t_alpha = stats.t.ppf(1-(0.2/2), n-1) #80% confidence interval
        SE = stdv/np.sqrt(n)
        CImin = xbar - t_alpha*SE
        CImax = xbar + t_alpha*SE
        months[m][s]['mean_CI'] = [CImin,CImax]

In [189]:
#CALCULATION OF MEDIAN CI's

samples = 1000 #number of samples to take

for s in states:
    tempDF = amazon_fires.loc[amazon_fires['state'] == s] #get a temp dataframe only 
    
    #loop through each month
    for m in mnths:
        monthly_data = tempDF.loc[tempDF['month'] == m] #get only data related to month
        n = len(monthly_data[(monthly_data['month'] == m)]) #get the length of the dataframe
        monthly_data.reset_index(inplace = True) #reset index
        count = 0 #reset count
        medians = [] #reset medians array
        
        while count < samples:#take 1000 resamples
            draws = []
            draws = random.choices(monthly_data.number, k=n) #draw n choices
            med = np.median(draws) #get median of sample
            medians.append(med) #append to list
            count += 1 #increment count
            
        months[m][s]['median_CI'] = np.percentile(medians, [10,90]) #set median CI's to percentile range

In [190]:
#DONT CHANGE THIS. WE USE IT TO MAKE THE OUTPUT LEGIBLE FOR GRADING
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(months)

{'April': {'Acre': {'mean_CI': [0.7631702958582414, 3.336829704141758],
                    'median_CI': array([0., 1.])},
           'Alagoas': {'mean_CI': [7.979189204540319, 16.52081079545968],
                       'median_CI': array([ 4.5, 16. ])},
           'Amapa': {'mean_CI': [0.3412383897376437, 0.9587616102623564],
                     'median_CI': array([0., 0.])},
           'Amazonas': {'mean_CI': [6.9737040719879255, 12.226295928012075],
                        'median_CI': array([ 3.5, 12. ])},
           'Bahia': {'mean_CI': [93.8962665362685, 157.1037334637315],
                     'median_CI': array([ 76. , 121.5])},
           'Ceara': {'mean_CI': [2.695004965166694, 4.904995034833306],
                     'median_CI': array([2., 4.])},
           'Distrito Federal': {'mean_CI': [0.6266818228169977,
                                            1.8733181771830023],
                                'median_CI': array([0., 1.])},
           'Espirito Santo': {'mean_CI

In [191]:
#Given Test for the mean confidence intervals

rounded_mean_CI = [round(x, 2) for x in months['April']['Acre']['mean_CI']]
assert rounded_mean_CI == [0.76, 3.34], 'somethings wrong in the mean'

In [192]:
#Given test for the median confidence intervals. 
#Your code is probably correct if it passes this test, but since bootstrapping the medain is a stochastic process
#you may have this test fail. If it fails, run it a few times. 
#If it continues to fail, your code is probably incorrect.

low_median_CI = months['April']['Acre']['median_CI'][0]
high_median_CI = months['April']['Acre']['median_CI'][1]
assert -1 <= low_median_CI <= 1 and 0 <= high_median_CI <= 3, 'somethings wrong in the median'

#### Part E: Where Do The Firefighters Go?
Now, we'll determine which state the Brazilian government should assign it's fire fighters to. For each month of the year, you should perform the folllowing selection process:
1. Find the state with the highest CI for the median for this month (it's easiest and ok to just use the upper bound here). 
2. Find any states that have a median CI that overlaps with the highest CI foud in step 1. If no states overlap with the highest CI found in step 1, then use that stat. 
3. If overlapping confidence intervals are found on the median, we'll use the CI for the mean to break ties.
4. Out of the states with overlapping CIs for median (every state in part 3), find the state with the highest mean CI. 
5. Determine if any of the states from part 3 have a mean CI that overlaps with the state found in step 4. 
6. If no state overlap with the state found in part 4, then just use that state. If other states have overlapping mean CIs too, then we'll split up the firefighters and assign some of them to every state that has both an overlapping median and mean CI with the state that has the highest median CI.

Once you've used the selection process above, use a markdown table to display a list of each state that recieves  some of the firefighters for each month.

In [214]:
#loop through each month
for m in mnths:
    print ("Month:", m)
    print ("")
    high_CI = months[m]['Acre']['median_CI'][1] #set the highest to first CI
    low_CI = months[m]['Acre']['median_CI'][0] #set the lowest to first CI
    name = "Acre"
    firefighters = 500 #number of firefighters to be allocated
    
    #Step 1: loop through each state and find the highest median
    for s in states: 
        if months[m][s]['median_CI'][1] > high_CI:
            high_CI = months[m][s]['median_CI'][1] #reset high
            low_CI = months[m][s]['median_CI'][0] #reset low
            name = s #reset name
    
    list_overlap_median = []
    
    #Step 2: find overlapping CI's
    for s in states:
        if s != name:
            if months[m][s]['median_CI'][1] > low_CI:
                list_overlap_median.append(s)

    count = 0
    while count < 500:
        
        #Step 3: If list is empty, assign firefighters to state in Step 1, else add step 1 state to list
        if len(list_overlap_median) == 0:
            print (firefighters, "allocated to:", name)
            print ("")
            count += 500
            break
        else:
            list_overlap_median.append(name)
            
        #Step 4: Find the highest mean CI
        
        #set highest and lowest to the first name in the list
        name = list_overlap_median[0]
        high_mean = months[m][name]['mean_CI'][1]
        low_mean = months[m][name]['mean_CI'][0]
        
        list_overlap_mean = []
        
        #loop through the rest of the list to find the highest mean CI
        for s in list_overlap_median:
            if months[m][s]['mean_CI'][1] > high_mean:
                name = s
                high_mean = months[m][s]['mean_CI'][1]
                low_mean = months[m][s]['mean_CI'][0]
                
        #Step 5: find overlapping CI's
        for s in list_overlap_median:
            if s != name:
                if months[m][s]['mean_CI'][1] > low_mean:
                    list_overlap_mean.append(s)
        
        #Step 6: Allocate firefighters
        if len(list_overlap_mean) == 0:
            print (firefighters, "allocated to:", name)
            print ("")
            count += 500
            break
        else:
            list_overlap_mean.append(name)
            n = len(list_overlap_mean)
            ffs = firefighters/n
            for s in list_overlap_mean:
                print (ffs, "allocated to:", s)
                print ("")
                count += ffs

Month: January

250.0 allocated to: Roraima

250.0 allocated to: Para

Month: February

500 allocated to: Roraima

Month: March

500 allocated to: Roraima

Month: April

100.0 allocated to: Minas Gerais

100.0 allocated to: Sao Paulo

100.0 allocated to: Tocantins

100.0 allocated to: Roraima

100.0 allocated to: Bahia

Month: May

500 allocated to: Tocantins

Month: June

500 allocated to: Tocantins

Month: July

250.0 allocated to: Tocantins

250.0 allocated to: Maranhao

Month: August

500 allocated to: Para

Month: September

250.0 allocated to: Rondonia

250.0 allocated to: Para

Month: October

250.0 allocated to: Maranhao

250.0 allocated to: Para

Month: November

500 allocated to: Para

Month: December

500 allocated to: Para



A summary of the allocated firefighters, by month and by state, is presented below:


| Month  / State    | January | February | March | April | May | June | July | August | Sept. | Oct. | Nov. | Dec. |
|:------------------|---------|----------|-------|-------|-----|------|------|--------|-------|------|------|-----:|
| Acre              | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Alagoas           | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Amapa             | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Amazonas          | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Bahia             | -       | -        | -     | 100   | -   | -    | -    | -      | -     | -    | -    | -    |
| Ceara             | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Distrito Federal  | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Espirito Santo    | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Goias             | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Maranhao          | -       | -        | -     | -     | -   | -    | 250  | -      | -     | 250  | -    | -    |
| Minas Gerais      | -       | -        | -     | 100   | -   | -    | -    | -      | -     | -    | -    | -    |
| Para              | 250     | -        | -     | -     | -   | -    | -    | 500    | 250   | 250  | 500  | 500  |
| Pernambuco        | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Piau              | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Rondonia          | -       | -        | -     | -     | -   | -    | -    | -      | 250   | -    | -    | -    |
| Roraima           | 250     | 500      | 500   | 100   | -   | -    | -    | -      | -     | -    | -    | -    |
| Santa Catarina    | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Sao Paulo         | -       | -        | -     | 100   | -   | -    | -    | -      | -     | -    | -    | -    |
| Sergipe           | -       | -        | -     | -     | -   | -    | -    | -      | -     | -    | -    | -    |
| Tocantins         | -       | -        | -     | 100   | 500 | 500  | 250  | -      | -     | -    | -    | -    |

